# Matrices & Probabilities as Graphs | 

[Tai-Danae Bradley](https://www.math3ma.com/about) wrote a nice tutorial on [viewing matrices & probabilities as graphs](https://www.math3ma.com/blog/matrices-probability-graphs) in March 2019. 
I'm going to replicate a lot of it here with the view of comparing a graph implementation with the various alternate implementations.

Julia's standard library has a [number of specialized datastructures](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#Special-matrices) for [linear algebra](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/) as this permits the specialization of routines (for each particular matrix types) whereby computational overhead can be minimised. I have no doubt that researchers have spent enormous amounts of time on these problems, yet I find the optimization of the representation of a graph rather intriguing.

Perhaps this is because most matrix computation is focused on two dimensions - X,Y - which in general only require the subset of graphs that are bi-partite. 

the matrix:
```
M = [
    [-1, 0],
    [ 4,-3],
    [ 0, 2]
]
```
is clearly more familiar in plain text form than the graph representation:
```
g = {
    'x1': {'y1': -1          },
    'x2': {'y1':  4, 'y2': -3},
    'x3': {          'y2':  2}
}
```



However when density of the matrix drops, the number of zeros in conventional matrix redundant:
```
M = [
    [10, 20,  0,  0,  0,  0]
    [ 0, 30,  0, 40,  0,  0]
    [ 0,  0, 50, 60, 70,  0]
    [ 0,  0,  0,  0,  0, 80]
] (24 elements)
```

```
g = {
    'x1': {'y1': 10, 'y2': 20                                       },
    'x2': {          'y2': 30,           'y4': 40                   },
    'x3': {                    'y3': 50, 'y4': 60, 'y5': 70         },
    'x4': {                                                 'y6': 80}
} (20 elements)
```



The compressed sparse matrix form `M = [ [data], [row], [col] ]` is probably compact enough for the general case, even though this example doesn't do it any favour:
```
M = [
    [10, 20, 30, 40, 50, 60, 70, 80],
    [ 0,  0,  1,  1,  2,  2,  2,  3],
    [ 0,  1,  1,  3,  2,  3,  4,  5],
] (24 elements)
```

The linear address style where the width of the matrix is declared as `M = [ dimensionality , [values], [indices] ]`, e.g.:

```
M = [ 
    (4, 6), 
    [10, 20, 30, 40, 50, 60, 70, 80],
    [ 0,  1,  7,  9, 14, 15, 16, 23]
] (18 elements)
```

may at first seem attractive, but additional computation is required to determine locations:

- row of `60`: `15 // 6 = 2`
- col of `60`: `15 - row * 6 = 3`

So where am I at? I think using graphs to `explain` matrix operations is great, but when it comes to computation the compressed sparse matrix will win.